In [0]:
%pip install transformers torch

Python interpreter will be restarted.
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-87862528-353d-4a6a-b788-60635435afd5
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
# to summerise the feedback by using Hugging face Opensource Model using transformers load summeriser
from transformers import pipeline
summarizer = pipeline("summarization",model="facebook/bart-large-cnn") 
'''
famous open source models:
BART: facebook/bart-large-cnn
T5: t5-small or t5-base
Pegasus: google/pegasus-large
'''
'''
import openai

# Set your OpenAI API key
openai.api_key = 'your_api_key'

def summarize_feedback(feedback_text):
    prompt = f"Summarize the following feedback: {feedback_text}"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message['content']
'''

/local_disk0/.ephemeral_nfs/envs/pythonEnv-87862528-353d-4a6a-b788-60635435afd5/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Out[30]: '\nfamous open source models:\nBART: facebook/bart-large-cnn\nT5: t5-small or t5-base\nPegasus: google/pegasus-large\n'

In [0]:
# function for summarization
def summarize_feedback(feedback):
    return summarizer(feedback,max_length=30,min_length=5,do_sample=False)


In [0]:
feedback_df = spark.read.csv("/FileStore/tables/customer_feedback.csv",header=True,inferSchema=False)
display(feedback_df)

FeedbackID,CustomerID,FeedbackText
1,1001,"I love the new features of the app, but the loading speed needs improvement."
2,1002,"Great customer service! However, the product quality could be better."
3,1003,"The interface is user-friendly, but I encountered a few bugs."
4,1004,Excellent experience! Will recommend it to others.
5,1005,Not satisfied with the product; it did not meet my expectations.


In [0]:
summaries=[]
for row in feedback_df.collect():
    feedbackText = row.FeedbackText
    # print(feedbackText)
    res = summarize_feedback(feedbackText)
    # print(res[0].get('summary_text'))
    summarytext=res[0].get('summary_text')
    summaries.append((row.FeedbackID,summarytext))

Your max_length is set to 30, but your input_length is only 18. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)
Your max_length is set to 30, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 30, but your input_length is only 16. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)
Your max_length is set to 30, but your input_length is only 11. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)
Your max_len

In [0]:
res_df = spark.createDataFrame(summaries,["FeedbackId","Summary"])
display(res_df)

FeedbackId,Summary
1,"The app has a lot of new features, but the loading speed needs improvement."
2,"Great customer service! However, the product quality could be better."
3,"The interface is user-friendly, but I encountered a few bugs."
4,"""Excellent experience! Will recommend it to others. Excellent experience!"""
5,"""Not satisfied with the product; it did not meet my expectations"""
